# Gold Layer — Business Metrics

The Gold layer contains business-ready aggregated datasets designed for analytics and dashboards.

Key Gold tables include:

- daily_sales_metrics: Daily revenue and user activity

- product_performance: Product-level sales performance

- user_behavior_metrics: User engagement and spending behavior

These tables are optimized using Delta Lake and Z-Ordering for high-performance SQL analytics via Databricks SQL Warehouse.

#  Purpose of Gold Layer 

Gold tables represent aggregated, business-level metrics optimized for:

- Dashboards 

- Decision-making

- Executive & stakeholder consumption

##### Gold answers “What happened?”, not “What data do we have?”

## Create Gold Schema

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS e_commerce_capstone.gold;

## GOLD TABLE 1 — DAILY SALES METRICS

In [0]:
%sql

-- How much revenue and how many purchases happen per day?

CREATE OR REPLACE TABLE e_commerce_capstone.gold.daily_sales_metrics
USING DELTA
PARTITIONED BY (event_date)
AS
SELECT
  DATE(event_time)            AS event_date,
  COUNT(*)                    AS total_events,
  round(SUM(price) )                 AS total_revenue,
  COUNT(DISTINCT user_id)     AS unique_users
FROM e_commerce_capstone.silver.events_clean
WHERE event_type = 'purchase'
GROUP BY DATE(event_time);


num_affected_rows,num_inserted_rows


##### Performance Optimization


- Gold tables are partitioned by event_date to enable efficient date-based filtering.

- Z-Ordering is applied only on non-partition columns such as product_id and revenue to improve data skipping.

- Attempted Z-Ordering on partition columns is intentionally avoided as Delta Lake handles pruning automatically.

In [0]:
%sql
OPTIMIZE e_commerce_capstone.gold.daily_sales_metrics
ZORDER BY (total_revenue);


path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 211, List(minCubeSize(107374182400), List(0, 0), List(211, 277593), 0, List(0, 0), 0, null), null, 0, 0, 211, 211, false, 0, 0, 1769855571412, 1769855574174, 8, 0, null, List(0, 0), null, 4, 4, 0, 0, null)"


## GOLD TABLE 2 — PRODUCT PERFORMANCE

In [0]:
%sql

--Which products generate the most revenue?

CREATE OR REPLACE TABLE e_commerce_capstone.gold.product_performance
USING DELTA
AS
SELECT
  product_id,
  COUNT(*)                AS total_purchases,
  ROUND(SUM(price), 2)    AS total_revenue,
  ROUND(AVG(price), 2)    AS avg_price

FROM e_commerce_capstone.silver.events_clean
WHERE event_type = 'purchase'
GROUP BY
  DATE(event_time),
  product_id;



num_affected_rows,num_inserted_rows


#####Optimize for dashboard queries

In [0]:
%sql
OPTIMIZE e_commerce_capstone.gold.product_performance
ZORDER BY (product_id);


path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 13640574), 0, List(0, 0), 0, null), null, 0, 0, 1, 1, false, 0, 0, 1769855616815, 1769855618790, 8, 0, null, List(0, 0), null, 4, 4, 0, 0, null)"


## GOLD TABLE 3 — USER BEHAVIOR METRICS

In [0]:
%sql

--How engaged are users?
CREATE OR REPLACE TABLE e_commerce_capstone.gold.user_behavior_metrics
AS
SELECT
    user_id,

    COUNT(*) AS total_events,

    -- count only purchase events
    SUM(CASE WHEN is_purchase = 1 THEN 1 ELSE 0 END) AS purchase_count,

    -- sum price ONLY for purchases
    ROUND(
        SUM(CASE WHEN is_purchase = 1 THEN price ELSE 0 END),
        2
    ) AS total_spent

FROM e_commerce_capstone.silver.events_clean
GROUP BY user_id;



num_affected_rows,num_inserted_rows


### Validate Gold Tables

In [0]:
%sql
SHOW TABLES IN e_commerce_capstone.gold;


database,tableName,isTemporary
gold,daily_sales_metrics,false
gold,product_performance,false
gold,user_behavior_metrics,false
gold,user_segment,false
gold,user_segment_summary,false


In [0]:
%sql
SELECT * FROM e_commerce_capstone.gold.daily_sales_metrics LIMIT 10;


event_date,total_events,total_revenue,unique_users
2019-10-04,27041,8623684.0,19214
2019-10-08,23072,6819832.0,16590
2019-10-22,25385,7914693.0,18290
2020-02-25,28063,8813543.0,19296
2020-03-05,44406,1.3489015E7,30488
2020-03-06,43521,1.3207019E7,29982
2020-03-29,21294,6369797.0,14921
2019-12-29,49597,1.4346608E7,34346
2020-01-13,27525,8654978.0,18807
2020-02-22,28946,8737562.0,19930


In [0]:
%sql
SELECT * FROM e_commerce_capstone.gold.product_performance ORDER BY total_revenue DESC LIMIT 10;


product_id,total_purchases,total_revenue,avg_price
1005115,3696,3500717.0,947.16
1005115,3164,2682297.59,847.76
1005115,2449,2126295.27,868.23
1005115,2220,1926128.55,867.63
1005115,2143,1836345.44,856.9
1005115,2082,1798316.49,863.74
1005115,2068,1780262.23,860.86
1005115,2175,1766479.62,812.17
1005105,1454,1738141.2,1195.42
1005115,1814,1729344.86,953.33


In [0]:
%sql
SELECT * FROM e_commerce_capstone.gold.user_behavior_metrics LIMIT 10;

user_id,total_events,purchase_count,total_spent
532749124,121,0,0.0
545348348,69,0,0.0
514443726,1942,4,249.16
542625372,266,0,0.0
512518970,331,12,7296.03
518359495,29,0,0.0
530277278,60,0,0.0
534329190,195,3,435.25
562539824,571,1,343.39
528344547,232,2,385.57
